- 载入sogou语料库生成中文的单字向量

In [1]:
import numpy as np
w2v_weight = np.load('w2v_weight_sogou.word.npy')

In [2]:
import json
f=open("word2idx_sogou.word.json","r")
for line in f:
    word2idx = json.loads(line)
#f=open("idx2word_sogou.word.json","r")
#for line in f:
#    idx2word = json.loads(line)

- 训练数据读取和转换

In [3]:
seg_len = 32

In [4]:
input_file = 'train.txt'

In [5]:
import codecs
import sys
import math

def character_tagging(input_file):
    input_data = codecs.open(input_file, 'r', 'utf-8')
    global train_sent
    global train_y
    global max_len
    train_sent = []
    train_y = []
    max_len = 0
    for line in input_data.readlines():
        label_list = []
        word_list = line.strip().split()
       
        tot = 0
        for word in word_list:
            tot += len(word)
            if len(word) == 1:
                label_list.append(1)
            else:
                label_list.extend([0 for w in range(len(word)-1)])
                label_list.append(1)
        strtmp = ''.join(word_list)
        L = len(label_list)
        t = max(L-seg_len, 0)
        for i in range(0,t,seg_len//4):
            if(i+seg_len>L-1):
                end = L
            else:
                end = i+seg_len    
            train_sent.append(strtmp[i:end])
            train_y.append(label_list[i:end])
        
        if tot>max_len:
            max_len = tot

character_tagging(input_file)

In [6]:
print(len(train_sent))
max_len = seg_len

1519878


In [7]:
w2v_weight = np.concatenate([np.zeros((3,300)), w2v_weight], axis=0)

In [8]:
tot_unk = []
def sent2num(sentence):
    global word2idx
    global tot_unk
    predict_word_num = []
    for w in sentence:
        if w in word2idx:
            predict_word_num.append(word2idx[w])
        else:
            predict_word_num.append(word2idx['UNK'])
            tot_unk.append(w)
    return predict_word_num

In [9]:
train_x = []
for i in range(len(train_sent)):
    train_x.append(sent2num(train_sent[i]))


In [10]:
from keras.preprocessing import sequence
train_x = sequence.pad_sequences(train_x, maxlen=max_len, padding='post', value=0)
train_y = sequence.pad_sequences(train_y, maxlen=max_len, padding='post', value=0).reshape(-1,seg_len,1)

Using TensorFlow backend.


In [11]:
print(train_x.shape)
print(train_y.shape)

(1519878, 32)
(1519878, 32, 1)


- 训练模型

In [12]:
from __future__ import absolute_import
from __future__ import print_function

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Reshape, Flatten ,Dropout
from keras.regularizers import l1,l2
from keras.layers.convolutional import Convolution2D, MaxPooling2D,MaxPooling1D

In [20]:
train_X = train_x
train_Y = train_y

In [21]:
batch_size = 1024
maxfeatures = w2v_weight.shape[0] # 词典大小
print(w2v_weight.shape)

(364993, 300)


In [22]:
from keras import backend as K
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

In [23]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, TimeDistributed, Masking, MaxPool1D
print('Build model...')
model = Sequential()
model.add(Embedding(maxfeatures, 300, weights=[w2v_weight], input_length=max_len, mask_zero=True, trainable=False)) 
model.add(Bidirectional(LSTM(64, return_sequences=True,implementation=2, activation='relu'), merge_mode='sum'))
model.add(TimeDistributed(Dense(10, activation='relu')))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', recall, precision, fbeta_score])

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 32, 300)           109497900 
_________________________________________________________________
bidirectional_3 (Bidirection (None, 32, 64)            186880    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 32, 10)            650       
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 10)            0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 32, 1)             11        
Total params: 109,685,441
Trainable params: 109,685,441
Non-trainable params: 0
_________________________________________________________________


In [24]:
from keras.callbacks import Callback
class EarlyStopping(Callback):
    def __init__(self, patience=0, verbose=0):
        super(Callback, self).__init__()

        self.patience = patience
        self.verbose = verbose
        self.best_val_loss = -1
        self.wait = 0

    def on_epoch_end(self, epoch, logs={}):
        if not self.params['do_validation']:
            warnings.warn("Early stopping requires validation data!", RuntimeWarning)

        cur_val_loss = logs.get('val_fbeta_score')
        #cur_val_loss = logs.get('val_loss')
        if cur_val_loss > self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.wait = 0
        else:
            if self.wait <= self.patience:
                if self.verbose > 0:
                    print("Epoch %05d: early stopping" % (epoch))
                self.model.stop_training = True
            self.wait += 1

In [25]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import keras
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 
keras.backend.get_session().run(tf.global_variables_initializer())

In [26]:
print(train_X.shape)

(1519878, 32)


In [28]:
print("Train...")
earlystop = EarlyStopping(patience=0, verbose=5)
result = model.fit(train_X, train_Y, batch_size=batch_size, epochs=100, 
          validation_split=0.01, callbacks=[earlystop])

Train...


/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 109497900 elements. This may consume a large amount of memory.
  num_elements)


Train on 1504679 samples, validate on 15199 samples
Epoch 1/100
1504679/1504679 [==============================] - 243s 161us/step - loss: 0.2250 - acc: 0.9126 - recall: 0.9490 - precision: 0.9053 - fbeta_score: 0.9263 - val_loss: 0.1163 - val_acc: 0.9562 - val_recall: 0.9737 - val_precision: 0.9546 - val_fbeta_score: 0.9640
Epoch 2/100
1504679/1504679 [==============================] - 238s 158us/step - loss: 0.1103 - acc: 0.9606 - recall: 0.9832 - precision: 0.9499 - fbeta_score: 0.9663 - val_loss: 0.1048 - val_acc: 0.9654 - val_recall: 0.9790 - val_precision: 0.9642 - val_fbeta_score: 0.9715
Epoch 3/100
1504679/1504679 [==============================] - 239s 159us/step - loss: 0.0796 - acc: 0.9716 - recall: 0.9898 - precision: 0.9618 - fbeta_score: 0.9756 - val_loss: 0.1048 - val_acc: 0.9685 - val_recall: 0.9820 - val_precision: 0.9662 - val_fbeta_score: 0.9740
Epoch 4/100
1504679/1504679 [==============================] - 239s 159us/step - loss: 0.0648 - acc: 0.9770 - recall: 0.992

In [ ]:
model.save('model_data_aug_32_embedding.h5')

- 用test文本进行预测，评估效果

In [31]:
test_file = 'test.txt'
with open(test_file,'r') as f:
    lines = f.readlines()
    test_texts = [''.join(list(line.strip())) for line in lines]
print(test_texts[0])

扬帆远东做与中国合作的先行


In [32]:
def num2char(x):
    if(x>0.5):
        return ' '
    else:
        return ''
    
def predict_num(input_txt, predict_prob):
    result =  [w+num2char(l)  for w, l in zip(input_txt,predict_prob)]
    return ''.join(result) + '\n'

In [33]:

test_len = []
input_txt = []
test_num = []

for line in test_texts:
    input_txt.append(line)
    tmp = sent2num(line)
    for i in range(0,len(tmp),seg_len):
        if(i==len(tmp)):
            break
        if(i+seg_len<=len(tmp)):
            end = i+seg_len
        else:
            end = len(tmp)
        test_num.append(tmp[i:end])
    test_len.append(len(tmp))
test_num = sequence.pad_sequences(test_num, maxlen=max_len, padding='post')
test_num = np.array(test_num)
predict_prob = model.predict_proba(test_num)
predict_lable = model.predict_classes(test_num).reshape(-1)



In [36]:
test_output = []
z = int(0)
for i in range(len(test_len)):
    z = int(z)
    output_line = predict_num(input_txt[i], predict_lable[z:z+test_len[i]])
    z += ((test_len[i]+seg_len-1)//seg_len)*seg_len
    test_output.append(output_line)

In [37]:
with open('test_output.txt','w') as f:
    f.writelines(test_output)

- 最终使用perl脚本检验F值

In [38]:
! ./score train.txt test.answer.txt test_output.txt > deep_score_dense_seg_50_data_aug_embedding.txt

- 预测方式2

In [39]:

test_len = []
input_txt = []
test_num = []

for line in test_texts:
    input_txt.append(line)
    tmp = sent2num(line)
    if(len(tmp)<seg_len):
        test_num.append(tmp)
    else:
        for i in range(0,len(tmp)-seg_len+1):
            test_num.append(tmp[i:i+seg_len])
    test_len.append(len(tmp))
test_num = sequence.pad_sequences(test_num, maxlen=seg_len, padding='post')
test_num = np.array(test_num)
print(test_num.shape)
predict_prob = model.predict_proba(test_num)




(78784, 32)


In [40]:
test_output = []
z = 0
for i in range(len(test_len)):
    l = test_len[i]
    if(l<seg_len):
        tmp = predict_num(input_txt[i],predict_prob[z])
        test_output.append(tmp)
        z+=1
    else:
        tmp = [[]for j in range(l)]
        for j in range(0,l-seg_len+1):
            for k in range(seg_len):
                tmp[j+k].append(float(predict_prob[z+j][k]))
        for j in range(l):
            tmp[j]=sum(tmp[j])/len(tmp[j])
        tmp = predict_num(input_txt[i], tmp)
        test_output.append(tmp)
        z+=l-seg_len+1
        

In [41]:
with open('test_output.txt','w') as f:
    f.writelines(test_output)

In [42]:
! ./score train.txt test.answer.txt test_output.txt > deep_score_dense_seg_50_data_aug_embedding.txt